In [ ]:
#!python -m mitoinstaller install
#!python -m mitoinstaller install
!pip install chembl-downloader

# CHeMBL LPG from RDF 

In [6]:
%%time
from katana import remote
from katana.remote import import_data
from katana.remote.import_data import Operation
#client = katana.remote.Client(address="localhost:8080")
client = remote.Client(disable_version_check=True,address="localhost:8080")

chembl29_csv = client.create_graph(num_partitions=5)

print(f"Importing graph from CSV files into graph {chembl29_csv} ...")
import_data.csv(
    chembl29_csv,
    input_node_path="gs://katana-internal8/chemb29_rdf2lpg/lpg_chembl29/chemb29_rdf2lpg_lpg_chembl29_katana-graph-node-schema.tsv",
    input_edge_path="gs://katana-internal8/chemb29_rdf2lpg/lpg_chembl29/chemb29_rdf2lpg_lpg_chembl29_katana-graph-edge-schema.tsv",
    input_dir="gs://katana-internal8/chemb29_rdf2lpg/lpg_chembl29/",
    #have_headers=True,
    files_have_headers=False,
    data_delimiter="\t",
    schema_delimiter="\t",
    partition_policy="random-oec"
)

Importing graph from CSV files into graph Graph(2fxph5v26HdK9QUD1J5mKtNHKekouqNJUadGesbLfQi3,0) ...


          0/? [?op/s]

CPU times: user 2.92 s, sys: 290 ms, total: 3.21 s
Wall time: 7min 36s


# CHeMBL from csv's

### Notes: -- Erroring out in distributed mode.
###           Graph is created in a shared memory mode.(8 min)
###           See Data Engineering section if the csv files are to be recreated.

In [110]:
%%time

from katana import remote
from katana.remote import import_data
from katana.remote.import_data import Operation
#client = katana.remote.Client(address="localhost:8080")
client = remote.Client(disable_version_check=True,address="localhost:8080")

mapped = client.create_graph(num_partitions=5)

print(f"Importing graph from CSV files into graph {mapped} ...")
import_data.csv(
    mapped,
    input_node_path="gs://katana-internal5/chembl29/csv/chembl29_nodes.txt",
    input_edge_path="gs://katana-internal5/chembl29/csv/chembl29_edge_list.csv",
    input_dir="gs://katana-internal5/chembl29/csv/",
    #have_headers=True,
    files_have_headers=True,
    data_delimiter="\t",
    schema_delimiter="|",
    partition_policy="random-oec"
)

Importing graph from CSV files into graph Graph(5tBKNWfTrC2SxT1rgNoTM7j27GLMBY2efzZDaWjwuuMf,0) ...


          0/? [?op/s]

OperationError: import 8XYtXxMLMekj9cDcdjFFubpYdiCuErKpoJvayi5PfeJG-23TpGqjTwtdhVq7Js error: (../../server/src/ImportOperation.cpp:148): backtrace: (../../libgluon/src/CSVImport.cpp:1098): backtrace: reading input files: (../../libgluon/include/katana/BulkGraphOperation.h:621): backtrace: id property was not agreed on correctly (BulkGraphOperation.h:772)

# Data Engineering -- Creating csv files

### TO DO -- Needs to be converted to Dask UDF 

In [10]:
import dask.dataframe as dd
import dask.distributed
import pandas as pd

import chembl_downloader

katana-internal5/chembl29/csv/node/activities.csv

In [ ]:
from katana.remote.dask import dask_remote
from katana_enterprise.distributed.dask_integration import utils
from katana_enterprise.remote import Client, import_data
from katana_enterprise.schema import SchemaParser, get_schema_groups

In [102]:
input_csv_schema = pd.DataFrame({
                            'Tables': ['activities'
                                       ,'assays'
                                       ,'target_components'
                                       ,'target_dictionary'
                                       ,'component_sequences'
                                       ,'component_class'
                                       ,'protein_family_classification'
                                       ,'molecule_dictionary'
                                       ,'molecule_hierarchy'
                                       ,'compound_structures'
                                       ,'drug_indication'
                                       ,'compound_properties'
                                       ,'compound_records'
                                       ,'docs'
                                       ,'source'
                                       ,'activity_properties'
                                       ,'assay_parameters'],
                            'Columns': [['activity_id','doc_id','standard_value','pchembl_value','toid','activity_comment','standard_units','standard_type','data_validity_comment','standard_relation','potential_duplicate','molregno']
                                       ,['assay_id','confidence_score','description']
                                       ,['tid']
                                       ,['tid','chembl_id','target_type','organism','pref_name','species_group_flag','tax_id']
                                       ,['component_id','accession']
                                       ,['component_id']
                                       ,['protein_class_id','l1','l2','l3']
                                       ,['molregno','chembl_id','oral']
                                       ,['molregno','parent_molregno','active_molregno']
                                       ,['molregno','standard_inchi','standard_inchi_key','canonical_smiles']
                                       ,['drugind_id','record_id','molregno','max_phase_for_ind','mesh_id','mesh_heading','efo_id','efo_term']
                                       ,['molregno','mw_freebase','alogp','hba','hbd','psa','rtb','ro3_pass','num_ro5_violations','molecular_species','full_mwt','aromatic_rings','heavy_atoms','qed_weighted','mw_monoisotopic','full_molformula','hba_lipinski','hbd_lipinski','num_lipinski_ro5_violations']
                                       ,['record_id','compound_key','compound_name']
                                       ,['doc_id','journal','year','volume','issue','first_page','last_page','pubmed_id','doi','chembl_id','title','doc_type','authors','abstract']
                                       ,['src_id','src_description']
                                       ,['ap_id','standard_type','standard_value','standard_text_value']
                                       ,['assay_param_id','standard_type','standard_value','standard_text_value']
                                       ],
                            'Types':   [
                                        {'activity_id':'int','doc_id':'int','standard_value':'float','pchembl_value':'float','toid':'float','activity_comment':'string','standard_units':'string','standard_type':'string','data_validity_comment':'string','standard_relation':'string','potential_duplicate':'int','molregno':'int'}
                                       ,{'assay_id':'int','confidence_score':'int','description':'string'}
                                       , {'tid':'int'}
                                       , {'tid':'int','chembl_id':'string','target_type':'string','organism':'string','pref_name':'string','species_group_flag':'string','tax_id':'string'}
                                       ,{'component_id':'int','accession':'string'}
                                       ,{'component_id':'int'}
                                       ,{'protein_class_id':'int','l1':'string','l2':'string','l3':'string'}
                                       ,{'molregno':'int','chembl_id':'string','oral':'int'}
                                       ,{'molregno':'int','parent_molregno':'int','active_molregno':'int'}
                                       ,{'molregno':'int','standard_inchi':'string','standard_inchi_key':'string','canonical_smiles':'string'}
                                       ,{'drugind_id':'int','record_id':'int','molregno':'int','max_phase_for_ind':'int','mesh_id':'string','mesh_heading':'string','efo_id':'string','efo_term':'string'}
                                       ,{'molregno':'int','mw_freebase':'string','alogp':'string','hba':'string','hbd':'string','psa':'string','rtb':'string','ro3_pass':'string','num_ro5_violations':'string','molecular_species':'string','full_mwt':'string','aromatic_rings':'string','heavy_atoms':'string','qed_weighted':'string','mw_monoisotopic':'string','full_molformula':'string','hba_lipinski':'string','hbd_lipinski':'string','num_lipinski_ro5_violations':'string'}
                                       ,{'record_id':'int','compound_key':'string','compound_name':'string'}
                                       ,{'doc_id':'int64','journal':'string','year':'string','volume':'string','issue':'string','first_page':'string','last_page':'string','pubmed_id':'string','doi':'string','chembl_id':'string','title':'string','doc_type':'string','authors':'string','abstract':'string'}
                                       ,{'src_id':'int','src_description':'string'}
                                       ,{'ap_id':'int','standard_type':'string','standard_value':'float','standard_text_value':'string'}
                                       ,{'assay_param_id':'int','standard_type':'string','standard_value':'float','standard_text_value':'string'}
                                       ],
                            'RawFileLoc': ['activities.csv'
                                    ,'assays.csv'
                                    ,'target_components.csv'
                                    ,'target_dictionary.csv'                                   
                                    ,'component_sequences.csv'
                                    ,'component_class.csv'
                                    ,'protein_family_classification.csv'
                                    ,'molecule_dictionary.csv'
                                    ,'molecule_hierarchy.csv'
                                    ,'compound_structures.csv'
                                    ,'drug_indication.csv'
                                    ,'compound_properties.csv'
                                    ,'compound_records.csv'
                                    ,'docs.csv'
                                    ,'source.csv'
                                    ,'activity_properties.csv'
                                    ,'assay_parameters.csv']
                        })

edges_csv_schema = pd.DataFrame({
      'EdgeFileLoc': ['has_assay.csv'
                    ,'has_target.csv'
                    ,'has_activity.csv'
                    ,'has_component.csv'
                    ,'has_comp_seq.csv'
                    ,'has_class_comp.csv'
                    ,'has_protein_class.csv'
                    ,'has_molecule_hierarchy.csv'
                    ,'has_comp_struct.csv'
                    ,'has_drugind.csv'
                    ,'has_comp_prop.csv'
                    ,'has_comp_records.csv'
                    ,'has_docs.csv'
                    ,'has_source.csv'
                    ,'has_act_prop.csv'
                    ,'has_assay_param.csv'] ,  

    'EdgesSQL':[ '  SELECT activity_id,activities.assay_id,"has_assay" FROM activities  JOIN assays ON activities.assay_id = assays.assay_id '
                   , ' SELECT assays.tid, target_dictionary.tid ,"has_target" FROM assays JOIN target_dictionary ON assays.tid = target_dictionary.tid '
                   , ' SELECT activities.activity_id, molecule_dictionary.molregno ,"has_activity" FROM activities JOIN molecule_dictionary ON activities.molregno = molecule_dictionary.molregno '
                   , ' SELECT target_dictionary.tid , target_components.tid ,"has_component" FROM target_dictionary JOIN target_components ON target_dictionary.tid = target_components.tid '
                   , ' SELECT target_components.tid, component_sequences.component_id  ,"has_comp_seq" FROM target_components JOIN component_sequences ON component_sequences.component_id = target_components.component_id '
                   , ' SELECT target_components.tid, component_class.component_id  ,"has_class_comp" FROM target_components JOIN component_class ON target_components.component_id = component_class.component_id '
                   , ' SELECT component_class.component_id, protein_family_classification.protein_class_id  ,"has_protein_class" FROM component_class JOIN protein_family_classification ON component_class.protein_class_id = protein_family_classification.protein_class_id '
                   , ' SELECT molecule_dictionary.molregno, molecule_hierarchy.molregno ,"has_molecule_hierarchy" FROM molecule_dictionary JOIN molecule_hierarchy ON molecule_dictionary.molregno = molecule_hierarchy.molregno'
                   , ' SELECT molecule_hierarchy.molregno,compound_structures.molregno  ,"has_comp_struct" FROM molecule_hierarchy JOIN compound_structures ON molecule_hierarchy.parent_molregno = compound_structures.molregno'
                   , ' SELECT MOLECULE_DICTIONARY.molregno, DRUG_INDICATION.drugind_id  ,"has_drugind" FROM MOLECULE_DICTIONARY JOIN DRUG_INDICATION ON MOLECULE_DICTIONARY.molregno == DRUG_INDICATION.molregno '
                   , ' SELECT MOLECULE_DICTIONARY.molregno , COMPOUND_PROPERTIES.molregno  ,"has_comp_prop" FROM MOLECULE_DICTIONARY JOIN COMPOUND_PROPERTIES ON MOLECULE_DICTIONARY.molregno == COMPOUND_PROPERTIES.molregno '
                   , ' SELECT activities.activity_id, COMPOUND_RECORDS.record_id  ,"has_comp_records" FROM activities JOIN COMPOUND_RECORDS ON activities.record_id = COMPOUND_RECORDS.record_id '
                   , ' SELECT COMPOUND_RECORDS.record_id, DOCS.doc_id ,"has_docs" FROM COMPOUND_RECORDS JOIN DOCS ON DOCS.doc_id = COMPOUND_RECORDS.doc_id '
                   , ' SELECT COMPOUND_RECORDS.src_id, SOURCE.src_id ,"has_source" FROM COMPOUND_RECORDS JOIN SOURCE ON SOURCE.src_id = COMPOUND_RECORDS.src_id '
                   , ' SELECT ACTIVITIES.activity_id , actp.ap_id  ,"has_act_prop" FROM ACTIVITIES JOIN activity_properties actp ON ACTIVITIES.activity_id = actp.activity_id '
                   , ' SELECT assays.assay_id, ap.assay_param_id   ,"has_assay_param" FROM ASSAYS JOIN assay_parameters ap ON ASSAYS.assay_id = ap.assay_id '
                  ]
          })

# Setting up Paths

In [85]:
BUCKET_NAME = "katana-internal5"
FOLDER_NAME = "chembl29"

DATA_FOLDER = "/".join(["gs:/", BUCKET_NAME, FOLDER_NAME])

NODE_LIST_FILE = "/".join(["gs:/", BUCKET_NAME, FOLDER_NAME, "csv/nodelist.txt"])
EDGE_LIST_FILE = "/".join(["gs:/", BUCKET_NAME, FOLDER_NAME, "csv/edgelist.txt"])

NODES_FOLDER = "/".join(["gs:/", BUCKET_NAME, FOLDER_NAME, "csv/node/"])
EDGES_FOLDER = "/".join(["gs:/", BUCKET_NAME, FOLDER_NAME, "csv/edge/"])

INPUT_FILES = "/".join(["gs:/", BUCKET_NAME, FOLDER_NAME, "raw/"])


In [96]:
ind=3
sql = 'SELECT * FROM {0}'.format(input_csv_schema['Tables'][ind])
df = chembl_downloader.query(sql)
print(df.columns)
print(input_csv_schema['Columns'][ind])
print(input_csv_schema['Types'][ind])
exp_df=df[input_csv_schema['Columns'][ind]]

exp_df.astype(input_csv_schema['Types'][ind])
exp_df.dtypes

Index(['tid', 'target_type', 'pref_name', 'tax_id', 'organism', 'chembl_id',
       'species_group_flag'],
      dtype='object')
['tid', 'chembl_id', 'target_type', 'organism', 'pref_name', 'species_group_flag', 'tax_id']
{'tid': 'int', 'chembl_id': 'string', 'target_type': 'string', 'organism': 'string', 'pref_name': 'string', 'species_group_flag': 'string', 'tax_id': 'string'}


tid                     int64
chembl_id              object
target_type            object
organism               object
pref_name              object
species_group_flag      int64
tax_id                float64
dtype: object

In [82]:
ind=14
output_file = EDGES_FOLDER + edges_csv_schema['EdgeFileLoc'][ind]
sql = '{0}'.format(edges_csv_schema['EdgesSQL'][ind])
print(output_file)
print(sql)
#df = chembl_downloader.query(sql)
#df.to_csv(output_file)
print(ind)

gs://katana-internal5/chembl29/csv/edge/has_act_prop.csv
 SELECT ACTIVITIES.activity_id , actp.ap_id  ,"has_act_prop" FROM ACTIVITIES JOIN activity_properties actp ON ACTIVITIES.activity_id = actp.activity_id 
14


# NODE and EDGE File Creation

In [103]:
with open('chembl29.config') as f:
    contents = f.read()
#input_csv_schema=input_csv_schema[13:14]
for ind in input_csv_schema.index:
    output_file = NODES_FOLDER + input_csv_schema['RawFileLoc'][ind]
    sql = 'SELECT * FROM {0}'.format(input_csv_schema['Tables'][ind])
    df = chembl_downloader.query(sql)
    exp_df=df[input_csv_schema['Columns'][ind]]
    exp_df.astype(input_csv_schema['Types'][ind])
    exp_df.dtypes
    exp_df.to_csv(output_file,index=False,sep='\t')
    print('{0} {1}'.format(ind, output_file))
    #print(exp_df.head(2))
    #edges_csv_schema=edges_csv_schema[14:17]
for ind in edges_csv_schema.index:
    output_file = EDGES_FOLDER + edges_csv_schema['EdgeFileLoc'][ind]
    sql = '{0} LIMIT 100'.format(edges_csv_schema['EdgesSQL'][ind])
    df = chembl_downloader.query(sql)
    df.to_csv(output_file,index=False,sep='\t')
    print(ind)
    

0 gs://katana-internal5/chembl29/csv/node/activities.csv
1 gs://katana-internal5/chembl29/csv/node/assays.csv
2 gs://katana-internal5/chembl29/csv/node/target_components.csv
3 gs://katana-internal5/chembl29/csv/node/target_dictionary.csv
4 gs://katana-internal5/chembl29/csv/node/component_sequences.csv
5 gs://katana-internal5/chembl29/csv/node/component_class.csv
6 gs://katana-internal5/chembl29/csv/node/protein_family_classification.csv
7 gs://katana-internal5/chembl29/csv/node/molecule_dictionary.csv
8 gs://katana-internal5/chembl29/csv/node/molecule_hierarchy.csv
9 gs://katana-internal5/chembl29/csv/node/compound_structures.csv
10 gs://katana-internal5/chembl29/csv/node/drug_indication.csv
11 gs://katana-internal5/chembl29/csv/node/compound_properties.csv
12 gs://katana-internal5/chembl29/csv/node/compound_records.csv
13 gs://katana-internal5/chembl29/csv/node/docs.csv
14 gs://katana-internal5/chembl29/csv/node/source.csv
15 gs://katana-internal5/chembl29/csv/node/activity_propertie

In [ ]:
#edges_csv_schema=edges_csv_schema[14:17]
for ind in edges_csv_schema.index:
    output_file = EDGES_FOLDER + edges_csv_schema['EdgeFileLoc'][ind]
    sql = '{0}'.format(edges_csv_schema['EdgesSQL'][ind])
    df = chembl_downloader.query(sql)
    df.to_csv(output_file,index=False,sep='\t')
    print(ind)


# Schema

In [ ]:
node_schema = pd.DataFrame({'nodes':['activity_id:ID(activities)|standard_value:float|pchembl_value:float|toid:float|activity_comment:string|standard_units:string|data_validity_comment:string|standard_relation:string|potential_duplicate:int|record_id:int|molregno:int'
                                     ,'assay_id:ID(assays)|confidence_score:float|assay_description:string'
                                     ,'tid:ID(target_components)'
                                     ,'tid:ID(target_dictionary)|target_chembl_id:string|target_type:string|organism:string|pref_name:string|species_group_flag:string|tax_id:int'
                                     ,'component_id:ID(component_sequences)|accession:string
                                     ,'component_id:ID(component_class)
                                     ,'protein_class_id:ID(protein_family_classification)|l1:string|l2:string|l3:string
                                     ,'molregno:ID(molecule_dictionary)|chembl_id:string|oral:int
                                     ,'record_id:ID(molecule_hierarchy)|molregno:int|parent_molregno:int|active_molregno:int'
                                     ,'molregno:ID(compound_structures)|standard_inchi:string|standard_inchi_key:string|canonical_smiles:string'
                                     ,'drugind_id:ID(drug_indication)|record_id:string|molregno_di:string|max_phase_for_ind:int|mesh_id:string|mesh_heading:string|efo_id:string|efo_term:string'
                                     ,'molegno:ID(compound_properties)|mw_freebase:float|alogp:float|hba:int|hbd:int|psa:float|rtb:int|ro3_pass:string|num_ro5_violations:int|molecular_species:string|full_mwt:float|aromatic_rings:int|heavy_atoms:int|qed_weighted:float|mw_monoisotopic:float|full_molformula:string|hba_lipinski:int|hbd_lipinski:int|num_lipinski_ro5_violations:int
                                     ,'record_id:ID(compound_records)|compound_key:string|compound_name:string'
                                     ,'doc_id:ID(docs)|journal:string|year:int|volume:string|issue:string|first_page:string|last_page:string|pubmed_id:string|doi:string|doc_chembl_id:string|title:string|doc_type:string|authors:string|abstract:string'
                                     ,'src_id:ID(source)|src_description:string'
                                     ,'ap_id:ID(activity_properties)|actp_standard_type:string|actp_standard_value:float|actp_standard_text_value:string'
                                     ,'assay_param_id:ID(assay_parameter)|standard_type:string|standard_value:float|standard_text_value:string'
                                    ],
                            'node_loc': ['nodes/activities.csv'
                                    ,'nodes/assays.csv'
                                    ,'nodes/target_dictionary.csv'
                                    ,'nodes/molecule_dictionary.csv'
                                    ,'nodes/target_components.csv'
                                    ,'nodes/component_sequences.csv'
                                    ,'nodes/component_class.csv'
                                    ,'nodes/protein_family_classification.csv'
                                    ,'nodes/molecule_hierarchy.csv'
                                    ,'nodes/compound_structures.csv'
                                    ,'nodes/DRUG_INDICATION.csv'
                                    ,'nodes/COMPOUND_PROPERTIES.csv'
                                    ,'nodes/COMPOUND_RECORDS.csv'
                                    ,'nodes/DOCS.csv'
                                    ,'nodes/SOURCE.csv'
                                    ,'nodes/activity_properties.csv'
                                    ,'nodes/assay_parameters.csv']
                           })
                 
edge_schema = pd.DataFrame({'edges': [':START_ID(activities)|:END_ID(molecule_dictionary)|:TYPE'
                                     ,':START_ID(activities)|:END_ID(assays)|:TYPE'
                                     ,':START_ID(assays)|:END_ID(target_dictionary)|:TYPE'
                                     ,':START_ID(molecule_dictionary)|:END_ID(compound_structures)|:TYPE'
                                     ,':START_ID(molecule_dictionary)|:END_ID(compound_properties)|:TYPE'
                                     ,':START_ID(drug_indication)|:END_ID(molecule_dictionary)|:TYPE'
                                     ,':START_ID(target_components)|:END_ID(target_dictionary)|:TYPE'
                                     ,':START_ID(target_components)|:END_ID(component_sequences)|:TYPE'
                                     ,':START_ID(molecule_hierarchy)|:END_ID(molecule_dictionary)|:TYPE'
                                     ,':START_ID(activities)|:END_ID(compound_records)|:TYPE'
                                     ,':START_ID(compound_records)|:END_ID(docs)|:TYPE'
                                     ,':START_ID(compound_records)|:END_ID(source)|:TYPE'
                                     ,':START_ID(target_components)|:END_ID(component_class)|:TYPE'
                                     ,':START_ID(component_class)|:END_ID(protein_family_classification)|:TYPE'
                                     ,':START_ID(activities)|:END_ID(activity_properties)|:TYPE'
                                     ,':START_ID(assays)|:END_ID(assay_parameter)|:TYPE'
                                    ],
                            'edge_loc': ['edges/has_molecule_dict.csv'
                                    ,'edges/has_assay.csv'
                                    ,'edges/has_target.csv'
                                    ,'edges/has_molecule_csr.csv'
                                    ,'edges/has_molecule_compr.csv'
                                    ,'edges/has_molecule_drug.csv'
                                    ,'edges/has_target_comp.csv'
                                    ,'edges/has_comp_seq.csv'
                                    ,'edges/has_parent_molecule.csv'
                                    ,'edges/has_compound_records.csv'
                                    ,'edges/has_docs.csv'
                                    ,'edges/has_source.csv'
                                    ,'edges/has_activity_properties.csv'
                                    ,'edges/has_assay_parameters.csv'
                                        ]
                           })    


with open('readme.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

In [7]:
import mitosheet
mitosheet.sheet()

MitoWidget(analysis_data_json='{"analysisName": "UUID-b6b693d0-8386-4cb8-91d7-1ed33281957d", "code": {"imports…

In [ ]:
from mitosheet import *; register_analysis('UUID-b6b693d0-8386-4cb8-91d7-1ed33281957d')
    
# Imported chembl29.csv
import pandas as pd
chembl29 = pd.read_csv(r'chembl29.csv')

# Sorted first_page in chembl29 in descending order
chembl29 = chembl29.sort_values(by='first_page', ascending=False, na_position='last')
